In [1]:
import numpy as np
import pytest
import os
os.environ["GEOMETRICALGEBRA_NUMPY"] = "jax"

from geometricalgebra import cga3d
from geometricalgebra import cga2d

from geometricalgebra.vector import ga_numpy

import jax
from jax import numpy as jnp
from jax.scipy.optimize import minimize

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
class Robot:
    
    def __init__(self, link_lengths):
        self._link_lengths = link_lengths

    def forward(self, joints, ret_links=False):
        p = cga3d.Vector.from_identity()
        links = []
        for j, l in zip(joints, self._link_lengths):
            p = cga3d.Vector.from_rotator(j * cga3d.e_1 ^ cga3d.e_2) & p
            p = cga3d.Vector.from_translator(l * cga3d.e_1 ^ cga3d.e_inf) & p
            links.append(p.inverse())
        if ret_links:
            return p.inverse(), cga3d.Vector.stack(links)
        return p.inverse()
    
    def _distance_to_goal_pose(self, joints, goal_pose):
            test_pose = self.forward(joints)
            return - 2 * (test_pose.apply(cga3d.POSE_ORIGIN).scalar_product(goal_pose.apply(cga3d.POSE_ORIGIN)).sum()-3)
        
    def inverse(self, pose):
        result = minimize(self._distance_to_goal_pose, jnp.array([.2,.1,.1]), method="BFGS", tol=1e-8, args=(goal_pose,))
        if result.success and abs(result.fun) < 1e-8:
            return result.x
        raise ValueError("No solution found")
robot = Robot([1, 1, 1])

In [3]:
pose = robot.forward([1, 1, 1])
print(f"pose: {pose.to_pos_and_rot_vector()}")

pose: [ 0.86583703  1.89188842  0.          0.          0.         -3.        ]


In [4]:
goal_pose = cga3d.Vector.from_pos_and_rot_vector([-1.73701487,  2.21266479,  0,  0,  0, -1.2])
joints = robot.inverse(goal_pose)
print(f"joints: {joints}")

joints: [0.40000033 0.69999891 0.10000087]
